# Mortgage Underwriting AI Assistant Data Analysis

### Table of Contents

1. [Notebook setup](#Notebook-setup)
2. [ETL](#ETL)
  1. [Year](#Year)
  2. [Loan Limit](#Loan-limit)
  3. [Gender](#Gender)
  4. [PreApproval](#PreApproval)
  5. [Loan type](#Loan-type)
  6. [Loan purpose](#Loan-purpose)
  7. [Credit worthiness](#Credit-worthiness)

### Notebook setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
dataset = pd.read_csv('data.csv')

In [3]:
dataset.head()

,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,...,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
0,24890,2019,cf,Sex Not Available,nopre,type1,p1,l1,nopc,nob/c,...,EXP,758,CIB,25-34,to_inst,98.728814,south,direct,1,45.0
1,24891,2019,cf,Male,nopre,type2,p1,l1,nopc,b/c,...,EQUI,552,EXP,55-64,to_inst,NaN,North,direct,1,NaN
2,24892,2019,cf,Male,pre,type1,p1,l1,nopc,nob/c,...,EXP,834,CIB,35-44,to_inst,80.019685,south,direct,0,46.0
3,24893,2019,cf,Male,nopre,type1,p4,l1,nopc,nob/c,...,EXP,587,CIB,45-54,not_inst,69.376900,North,direct,0,42.0
4,24894,2019,cf,Joint,pre,type1,p1,l1,nopc,nob/c,...,CRIF,602,EXP,25-34,not_inst,91.886544,North,direct,0,39.0


Clean up column names

In [4]:
dataset.columns= dataset.columns.str.lower()

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148670 entries, 0 to 148669
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id                         148670 non-null  int64  
 1   year                       148670 non-null  int64  
 2   loan_limit                 145326 non-null  object 
 3   gender                     148670 non-null  object 
 4   approv_in_adv              147762 non-null  object 
 5   loan_type                  148670 non-null  object 
 6   loan_purpose               148536 non-null  object 
 7   credit_worthiness          148670 non-null  object 
 8   open_credit                148670 non-null  object 
 9   business_or_commercial     148670 non-null  object 
 10  loan_amount                148670 non-null  int64  
 11  rate_of_interest           112231 non-null  float64
 12  interest_rate_spread       112031 non-null  float64
 13  upfront_charges            10

### ETL

There are 34 features in the dataset. Analyze each feature setting the correct datatype, imputing missing values as needed and dropping features as needed

#### ID

ID can be dropped...status will be the independent feature

In [6]:
dataset.drop('id', inplace=True, axis=1)

In [7]:
dataset.head()

,year,loan_limit,gender,approv_in_adv,loan_type,loan_purpose,credit_worthiness,open_credit,business_or_commercial,loan_amount,...,credit_type,credit_score,co-applicant_credit_type,age,submission_of_application,ltv,region,security_type,status,dtir1
0,2019,cf,Sex Not Available,nopre,type1,p1,l1,nopc,nob/c,116500,...,EXP,758,CIB,25-34,to_inst,98.728814,south,direct,1,45.0
1,2019,cf,Male,nopre,type2,p1,l1,nopc,b/c,206500,...,EQUI,552,EXP,55-64,to_inst,NaN,North,direct,1,NaN
2,2019,cf,Male,pre,type1,p1,l1,nopc,nob/c,406500,...,EXP,834,CIB,35-44,to_inst,80.019685,south,direct,0,46.0
3,2019,cf,Male,nopre,type1,p4,l1,nopc,nob/c,456500,...,EXP,587,CIB,45-54,not_inst,69.376900,North,direct,0,42.0
4,2019,cf,Joint,pre,type1,p1,l1,nopc,nob/c,696500,...,CRIF,602,EXP,25-34,not_inst,91.886544,North,direct,0,39.0


#### Year

In [8]:
dataset['year'].unique()

array([2019], dtype=int64)

All values in the year column are for 2019. This column can be dropped.

In [9]:
dataset.drop('year', inplace=True, axis=1)

In [10]:
dataset.head()

,loan_limit,gender,approv_in_adv,loan_type,loan_purpose,credit_worthiness,open_credit,business_or_commercial,loan_amount,rate_of_interest,...,credit_type,credit_score,co-applicant_credit_type,age,submission_of_application,ltv,region,security_type,status,dtir1
0,cf,Sex Not Available,nopre,type1,p1,l1,nopc,nob/c,116500,NaN,...,EXP,758,CIB,25-34,to_inst,98.728814,south,direct,1,45.0
1,cf,Male,nopre,type2,p1,l1,nopc,b/c,206500,NaN,...,EQUI,552,EXP,55-64,to_inst,NaN,North,direct,1,NaN
2,cf,Male,pre,type1,p1,l1,nopc,nob/c,406500,4.56,...,EXP,834,CIB,35-44,to_inst,80.019685,south,direct,0,46.0
3,cf,Male,nopre,type1,p4,l1,nopc,nob/c,456500,4.25,...,EXP,587,CIB,45-54,not_inst,69.376900,North,direct,0,42.0
4,cf,Joint,pre,type1,p1,l1,nopc,nob/c,696500,4.00,...,CRIF,602,EXP,25-34,not_inst,91.886544,North,direct,0,39.0


#### Loan limit

In [11]:
dataset['loan_limit'].unique()

array(['cf', nan, 'ncf'], dtype=object)

There are some NaN values in this vector that will need to be cleaned up.

In [12]:
# Get a count of the number of applications missing the loan limits
dataset[dataset['loan_limit'].isnull()].shape[0]

3344

3344 applications are missing a loan limit designation

In [13]:
dataset['loan_limit'].fillna(0, inplace=True)

In [14]:
dataset['loan_limit'].unique()

array(['cf', 0, 'ncf'], dtype=object)

Encode conforming and non-conforming limits to 1 and 2

In [15]:
dataset['loan_limit'].replace({'cf': 1, 'ncf': 2}, inplace=True)

In [16]:
dataset['loan_limit'].unique()

array([1, 0, 2], dtype=int64)

#### Gender

In [17]:
dataset['gender'].unique()

array(['Sex Not Available', 'Male', 'Joint', 'Female'], dtype=object)

Encode 'Sex Not Available' to 0, Male to 1, Joint to 2, Female to 3

In [18]:
dataset['gender'].replace({'Sex Not Available': 0, 'Male': 1, 'Joint': 2, 'Female': 3}, inplace=True)

In [19]:
dataset['gender'].unique()

array([0, 1, 2, 3], dtype=int64)

#### PreApproval

Rename approv_in_adv to pre_approval

In [20]:
dataset.rename(columns={"approv_in_adv": "pre_approval"}, inplace=True)

In [21]:
dataset['pre_approval'].unique()

array(['nopre', 'pre', nan], dtype=object)

Encode nopre and nan to 0, pre to 1

In [22]:
dataset['pre_approval'].fillna(0, inplace=True)

In [23]:
dataset['pre_approval'].replace({'nopre': 0, 'pre': 1}, inplace=True)

In [24]:
dataset['pre_approval'].unique()

array([0, 1], dtype=int64)

#### Loan type

In [25]:
dataset['loan_type'].unique()

array(['type1', 'type2', 'type3'], dtype=object)

Drop the 'type' prefix, cast to int64

In [26]:
dataset['loan_type'] = dataset['loan_type'].str.replace('type', '').astype(np.int64)

In [27]:
dataset['loan_type'].unique()

array([1, 2, 3], dtype=int64)

#### Loan purpose

In [28]:
dataset['loan_purpose'].unique()

array(['p1', 'p4', 'p3', 'p2', nan], dtype=object)

Replace NaN with p0 then drop the 'p' prefix then cast as Int64

In [29]:
dataset['loan_purpose'].fillna('p0', inplace=True)

In [30]:
dataset['loan_purpose'].unique()

array(['p1', 'p4', 'p3', 'p2', 'p0'], dtype=object)

In [31]:
dataset['loan_purpose'] = dataset['loan_purpose'].str.replace('p', '').astype(np.int64)

In [32]:
dataset['loan_purpose'].unique()

array([1, 4, 3, 2, 0], dtype=int64)

#### Credit worthiness

In [33]:
dataset['credit_worthiness'].unique()

array(['l1', 'l2'], dtype=object)

Asses the correlation between credit worthiness and credit score

In [34]:
dataset[['credit_worthiness', 'credit_score']][dataset['credit_worthiness'] == 'l2']

,credit_worthiness,credit_score
10,l2,723
24,l2,846
69,l2,665
71,l2,663
89,l2,677
...,...,...
148598,l2,666
148621,l2,651
148632,l2,654
148636,l2,580


In [35]:
credit_worthiness_df = pd.DataFrame(dataset[['credit_worthiness', 'credit_score']])

In [36]:
# dataset['pre_approval'].replace({'nopre': 0, 'pre': 1}, inplace=True)
credit_worthiness_df['credit_worthiness'].replace({'l1': 0, 'l2': 2}, inplace=True)

In [37]:
credit_worthiness_df.head()

,credit_worthiness,credit_score
0,0,758
1,0,552
2,0,834
3,0,587
4,0,602


In [ ]:
plt.figure(figsize=(30, 20), dpi=100) 
_ = sns.heatmap(credit_worthiness_df, annot=True, annot_kws={"fontsize":12}, linecolor='white', \
            linewidth=1, fmt='.3f', cmap="viridis", cbar=False)